In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""Regeneration of lead synth from combined signal"""
import keras
from keras.layers import Dense, Dropout, PReLU
from keras.optimizers import Adam

from music_generator.basic.random import generate_dataset
from music_generator.basic.signalproc import SamplingInfo
from music_generator.musical.timing import Tempo
from music_generator.musical.scales import GenericScale
from music_generator.analysis.play import play_mono_as_stereo, play_array
from music_generator.basic.signalproc import mix_at
from music_generator.analysis import preprocessing

from music_generator.musical import scales
import numpy as np
from multiprocessing import Pool
from functools import partial

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io.wavfile import read
from music_generator.analysis import regen_models
from scipy.io import wavfile
from music_generator.analysis import jamdataset
import tensorflow as tf

from music_generator.analysis import regen_models
import importlib

/home/marcelra/Pycharmprojects/mg-desktop/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
sr = 44100
sampling_info = SamplingInfo(sr)
n_samples = 4096
fragment_length = 4096

In [4]:
model = regen_models.FftBranchesFilter(n_samples, learning_rate=1e-4, loss_fct='fft_loss').build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 4096)         0           lambda_1[0][0]                   
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 4096)         0           lambda_2[0][0]                   
__________________________________________________________________________________________________
dense_1 (D

In [ ]:
while True:

    jds = jamdataset.JamDataSet(fragment_length=fragment_length, n_samples_train=10000, n_samples_test=500)
    model.fit(jds.x_train, 
              jds.y_train, 
              epochs=5, 
              verbose=1,
              shuffle=True,
              batch_size=64,
              validation_data=[jds.x_test, jds.y_test])
    
    pred_model = regen_models.RegenModel(model=model, fragment_length=fragment_length)

    sr, dt = wavfile.read('../data/full-mix.wav')
    dt = dt / 2**16

    pred = pred_model.predict(dt)
    play_array(pred[20*sr:40*sr], do_wait_done=False)

/home/marcelra/Pycharmprojects/mg-desktop/venv/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 8.2731 - val_loss: 12.8164
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 4.8145 - val_loss: 12.1077
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 4.6760 - val_loss: 12.0648
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/step - loss: 4.5762 - val_loss: 11.0453
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 4.4503 - val_loss: 10.7447
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 4.2613 - val_loss: 10.5060
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 4.0789 - val_loss: 9.7023
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 3.8991 - val_loss: 9.1220
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/s

Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.9476 - val_loss: 4.8978
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.9355 - val_loss: 4.8596
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.9242 - val_loss: 4.8528
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.9419 - val_loss: 4.9182
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.9275 - val_loss: 4.8795
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.9146 - val_loss: 4.8413
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.9035 - val_loss: 4.8477
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.8927 - val_loss: 4.8344
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - 

10000/10000 [==============================] - 76s 8ms/step - loss: 1.5875 - val_loss: 4.5964
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.6527 - val_loss: 5.1292
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.6397 - val_loss: 5.1577
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.6302 - val_loss: 5.1545
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.6204 - val_loss: 5.1571
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.6122 - val_loss: 5.2036
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.6625 - val_loss: 4.6720
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.6463 - val_loss: 4.6586
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.63

Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.5324 - val_loss: 4.8669
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.5226 - val_loss: 4.8745
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.5156 - val_loss: 4.8411
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.5074 - val_loss: 4.8322
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.4920 - val_loss: 4.1148
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.4800 - val_loss: 4.0515
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.4718 - val_loss: 4.0893
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.4635 - val_loss: 4.0844
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.4551 - val_loss: 4.1445
Train on 10000 s

10000/10000 [==============================] - 76s 8ms/step - loss: 1.3924 - val_loss: 4.9152
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.3855 - val_loss: 4.9471
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.3822 - val_loss: 4.5157
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.3697 - val_loss: 4.6016
Epoch 3/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.3615 - val_loss: 4.5440
Epoch 4/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.3540 - val_loss: 4.5737
Epoch 5/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.3471 - val_loss: 4.5742
Train on 10000 samples, validate on 500 samples
Epoch 1/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.3840 - val_loss: 5.3682
Epoch 2/5
10000/10000 [==============================] - 76s 8ms/step - loss: 1.37

In [ ]:
# Audio(pred, rate=sr)

In [ ]:
# start_sec, end_sec = 20, 60

# pred = model_predict(model, input_track[start_sec*sr:end_sec*sr], fragment_length)
# play_array(input_track[start_sec*sr:end_sec*sr])
# play_array(pred)

# #%%

# x_pred = input_track[start_sec*sr:end_sec*sr]
# x_pred = x_pred[:(len(x_pred) // fragment_length) * fragment_length]
# x_pred = x_pred.reshape(-1, fragment_length)

# y_pred = model.predict(x_pred)
# play_array(y_pred)